In [1]:
import numpy as np
import matplotlib.pyplot as plt
import sys
import scipy
import scipy.optimize
import pyuvdata
import time

In [2]:
def get_test_data(use_autos=True):

    model_path = '/Users/ruby/Astro/FHD_outputs/fhd_rlb_model_GLEAM_Aug2021'
    model_use_model = True
    data_path = '/Users/ruby/Astro/FHD_outputs/fhd_rlb_model_GLEAM_Aug2021'
    data_use_model = True
    obsid = '1061316296'
    pol = 'XX'

    model_filelist = ['{}/{}'.format(model_path, file) for file in [
        'vis_data/{}_vis_{}.sav'.format(obsid, pol),
        'vis_data/{}_vis_model_{}.sav'.format(obsid, pol),
        'vis_data/{}_flags.sav'.format(obsid),
        'metadata/{}_params.sav'.format(obsid),
        'metadata/{}_settings.txt'.format(obsid),
        'metadata/{}_layout.sav'.format(obsid)
    ]]
    data_filelist = ['{}/{}'.format(data_path, file) for file in [
        'vis_data/{}_vis_{}.sav'.format(obsid, pol),
        'vis_data/{}_vis_model_{}.sav'.format(obsid, pol),
        'vis_data/{}_flags.sav'.format(obsid),
        'metadata/{}_params.sav'.format(obsid),
        'metadata/{}_settings.txt'.format(obsid),
        'metadata/{}_layout.sav'.format(obsid)
    ]]

    model = pyuvdata.UVData()
    print('Reading model...')
    model.read_fhd(model_filelist, use_model=model_use_model)
    
    # For testing, use one time and a few frequencies only
    use_time = model.time_array[200000]
    use_frequencies = model.freq_array[0, 100:200]
    #use_frequencies = data.freq_array[0, :]
    model.select(times=use_time, frequencies=use_frequencies)

    if not use_autos:  # Remove autocorrelations
        bl_lengths = np.sqrt(np.sum(model.uvw_array**2., axis=1))
        non_autos = np.where(bl_lengths > 0.01)[0]
        model.select(blt_inds=non_autos)

    if data_path != model_path or model_use_model != data_use_model:
        data = pyuvdata.UVData()
        print('Reading data...')
        data.read_fhd(data_filelist, use_model=data_use_model)
        print('Done.')
        data.select(times=use_time, frequencies=use_frequencies)
        if not use_autos:  # Remove autocorrelations
            bl_lengths = np.sqrt(np.sum(data.uvw_array**2., axis=1))
            non_autos = np.where(bl_lengths > 0.01)[0]
            data.select(blt_inds=non_autos)
    else:
        print('Using model for data')
        data = model.copy()
        
    # Need to check that the baseline ordering agrees between model and data

    return data, model

In [3]:
def initialize_cal(data):

    cal = pyuvdata.UVCal()
    cal.Nants_data = data.Nants_data
    cal.Nants_telescope = data.Nants_telescope
    cal.Nfreqs = data.Nfreqs
    cal.Njones = 1
    cal.Nspws = 1
    cal.Ntimes = 1
    cal.ant_array = np.arange(data.Nants_data)
    cal.antenna_names = data.antenna_names
    cal.antenna_numbers = data.antenna_numbers
    cal.cal_style = 'sky'
    cal.cal_type = 'gain'
    cal.channel_width = data.channel_width
    cal.freq_array = data.freq_array
    cal.gain_convention = 'divide'
    cal.history = ''
    cal.integration_time = np.mean(data.integration_time)
    cal.jones_array = np.array([-5])
    cal.spw_array = data.spw_array
    cal.telescope_name = data.telescope_name
    cal.time_array = np.array([np.mean(data.time_array)])
    cal.x_orientation = 'east'
    cal.gain_array = np.full((
        cal.Nants_data, cal.Nspws, cal.Nfreqs, cal.Ntimes, cal.Njones
    ), 1, dtype=complex)
    cal.flag_array = np.full((
        cal.Nants_data, cal.Nspws, cal.Nfreqs, cal.Ntimes, cal.Njones
    ), False, dtype=bool)
    cal.quality_array = np.full((
        cal.Nants_data, cal.Nspws, cal.Nfreqs, cal.Ntimes, cal.Njones
    ), 1., dtype=float)
    cal.ref_antenna_name = ''
    cal.sky_catalog = 'GLEAM_bright_sources'
    cal.sky_field = 'phase center (RA, Dec): ({}, {})'.format(
        np.degrees(np.mean(data.phase_center_app_ra)),
        np.degrees(np.mean(data.phase_center_app_dec))
    )

    if not cal.check():
        print('ERROR: UVCal check failed.')
        sys.exit(1)

    return cal

In [4]:
data, model = get_test_data(use_autos=False)

cal = initialize_cal(data)

# Create gains expand matrices
gains_exp_mat_1 = np.zeros((data.Nblts, cal.Nants_data), dtype=int)
gains_exp_mat_2 = np.zeros((data.Nblts, cal.Nants_data), dtype=int)
antenna_list = np.unique([data.ant_1_array, data.ant_2_array])
for baseline in range(data.Nblts):
    gains_exp_mat_1[
        baseline, np.where(antenna_list == data.ant_1_array[baseline])
    ] = 1
    gains_exp_mat_2[
        baseline, np.where(antenna_list == data.ant_2_array[baseline])
    ] = 1
    
# Test gains expand matrices
gains_exp_mat_1_test = np.sum(gains_exp_mat_1, axis=1)
gains_exp_mat_2_test = np.sum(gains_exp_mat_2, axis=1)
if np.max(gains_exp_mat_1_test) > 1 or np.max(gains_exp_mat_2_test) > 1:
    print('ERROR: Poorly defined gains expand matrices')
    sys.exit()

# Initialize gains
gain_init_noise = .1
#gain_init_noise = 0.
gains_init = (np.random.normal(
    1., gain_init_noise, size=(cal.Nants_data, cal.Nfreqs)
) + 1.j*np.random.normal(
    0., gain_init_noise, size=(cal.Nants_data, cal.Nfreqs)
))

# Define covariance matrix
cov_mat = np.identity(cal.Nfreqs)
cov_mat = np.repeat(cov_mat[np.newaxis, :, :], data.Nblts, axis=0)

Reading model...


Telescope location derived from obs lat/lon/alt values does not match the location in the layout file. Using the value from known_telescopes.
tile_names from obs structure does not match antenna_names from layout


Using model for data


In [5]:
gains = gains_init
model_vis = np.squeeze(model.data_array, axis=(1,3)) # Remove spw and pol axes
data_vis = np.squeeze(data.data_array, axis=(1,3))


In [6]:
def calculate_grad_real(gains_exp_mat_1, gains_exp_mat_2, gains, data_vis, model_vis, cov_mat):
    gains1_expanded = np.matmul(gains_exp_mat_1, gains)
    gains2_expanded = np.matmul(gains_exp_mat_2, gains)
    term1_part1 = -np.conj(gains1_expanded*model_vis)
    term2_part1 = -gains2_expanded*np.conj(model_vis)
    cost_term = data_vis - gains1_expanded*np.conj(gains2_expanded)*model_vis
    weighted_part2 = np.squeeze(np.matmul(cost_term[:, np.newaxis, :], cov_mat))
    term1 = np.matmul(gains_exp_mat_2.T, term1_part1*weighted_part2)
    term2 = np.matmul(gains_exp_mat_1.T, term2_part1*weighted_part2)
    grad = 2*np.real(term1 + term2)
    return grad

In [7]:
def calculate_grad_imag(gains_exp_mat_1, gains_exp_mat_2, gains, data_vis, model_vis, cov_mat):
    gains1_expanded = np.matmul(gains_exp_mat_1, gains)
    gains2_expanded = np.matmul(gains_exp_mat_2, gains)
    term1_part1 = -1.j*np.conj(gains1_expanded*model_vis)
    term2_part1 = 1.j*gains2_expanded*np.conj(model_vis)
    cost_term = data_vis - gains1_expanded*np.conj(gains2_expanded)*model_vis
    weighted_part2 = np.squeeze(np.matmul(cost_term[:, np.newaxis, :], cov_mat))
    term1 = np.matmul(gains_exp_mat_2.T, term1_part1*weighted_part2)
    term2 = np.matmul(gains_exp_mat_1.T, term2_part1*weighted_part2)
    grad = 2*np.real(term1 + term2)
    return grad

In [8]:
def calculate_negloglikelihood(gains_exp_mat_1, gains_exp_mat_2, gains, data_vis, model_vis, cov_mat):
    gains1_expanded = np.matmul(gains_exp_mat_1, gains)
    gains2_expanded = np.matmul(gains_exp_mat_2, gains)
    cost_term = data_vis - gains1_expanded*np.conj(gains2_expanded)*model_vis
    weighted_part2 = np.squeeze(np.matmul(cost_term[:, np.newaxis, :], cov_mat))
    negloglikelihood = np.real(np.sum(np.conj(cost_term)*weighted_part2))
    return negloglikelihood

In [9]:
# Test real gradient calculation
test_ant = 100
test_freq = 1
delta_gains = 0.0001
gains0 = np.copy(gains)
gains0[test_ant, test_freq] -= delta_gains/2.
gains1 = np.copy(gains)
gains1[test_ant, test_freq] += delta_gains/2.
negloglikelihood0 = calculate_negloglikelihood(gains_exp_mat_1, gains_exp_mat_2, gains0, data_vis, model_vis, cov_mat)
negloglikelihood1 = calculate_negloglikelihood(gains_exp_mat_1, gains_exp_mat_2, gains1, data_vis, model_vis, cov_mat)
grad_real = calculate_grad_real(gains_exp_mat_1, gains_exp_mat_2, gains, data_vis, model_vis, cov_mat)
print((negloglikelihood1-negloglikelihood0)/delta_gains)
print(grad_real[test_ant, test_freq])

13999.901879578829
13999.901875868476


In [10]:
# Test imaginary gradient calculation
test_ant = 100
test_freq = 1
delta_gains = 0.0001
gains0 = np.copy(gains)
gains0[test_ant, test_freq] -= 1j*delta_gains/2.
gains1 = np.copy(gains)
gains1[test_ant, test_freq] += 1j*delta_gains/2.
negloglikelihood0 = calculate_negloglikelihood(gains_exp_mat_1, gains_exp_mat_2, gains0, data_vis, model_vis, cov_mat)
negloglikelihood1 = calculate_negloglikelihood(gains_exp_mat_1, gains_exp_mat_2, gains1, data_vis, model_vis, cov_mat)
grad_imag = calculate_grad_imag(gains_exp_mat_1, gains_exp_mat_2, gains, data_vis, model_vis, cov_mat)
print((negloglikelihood1-negloglikelihood0)/delta_gains)
print(grad_imag[test_ant, test_freq])

-3450.250495225191
-3450.250514227262


In [11]:
def reformat_baselines_to_antenna_matrix(bl_array, gains_exp_mat_1, gains_exp_mat_2):
    (Nbls, Nants) = np.shape(gains_exp_mat_1)
    antenna_matrix = np.zeros_like(bl_array[0,], dtype=bl_array.dtype)
    antenna_matrix = np.repeat(np.repeat(antenna_matrix[np.newaxis,], Nants, axis=0)[np.newaxis,], Nants, axis=0)
    antenna_numbers = np.arange(Nants)
    antenna1_num = np.matmul(gains_exp_mat_1, antenna_numbers)
    antenna2_num = np.matmul(gains_exp_mat_2, antenna_numbers)
    for bl_ind in range(Nbls):
        antenna_matrix[antenna1_num[bl_ind], antenna2_num[bl_ind], ] = bl_array[bl_ind, ]
    return(antenna_matrix)

In [12]:
def calculate_hess(
    gains_exp_mat_1, gains_exp_mat_2, gains, data_vis, model_vis, cov_mat,
    Nants,
    Nfreqs,
    Nblts
):
    gains1_expanded = np.matmul(gains_exp_mat_1, gains)
    gains2_expanded = np.matmul(gains_exp_mat_2, gains)
    
    gains1_times_model = gains1_expanded*model_vis
    gains2_times_conj_model = gains2_expanded*np.conj(model_vis)
    
    term1 = (
        gains1_times_model[:, np.newaxis, :]
        *gains2_times_conj_model[:, :, np.newaxis]
        *np.conj(cov_mat)
    )
    term1 = reformat_baselines_to_antenna_matrix(term1, gains_exp_mat_1, gains_exp_mat_2)
    term1 = np.transpose(term1, (1, 0, 2, 3))
    term2 = (
        gains2_times_conj_model[:, np.newaxis, :]
        *gains1_times_model[:, :, np.newaxis]
        *cov_mat
    )
    term2 = reformat_baselines_to_antenna_matrix(term2, gains_exp_mat_1, gains_exp_mat_2)
    combined_terms = 2*(term1 + term2)
    
    # hess elements are antenna_c, antenna_d, freq_f0, freq_f1, and real/imag pair
    # The real/imag pairs are in order real-real, real-imag, and imag-imag
    hess = np.zeros((Nants, Nants, Nfreqs, Nfreqs, 3), dtype=float)
    hess[:, :, :, :, 0] = np.real(combined_terms)
    hess[:, :, :, :, 1] = np.imag(combined_terms)
    hess[:, :, :, :, 2] = -np.real(combined_terms)

    cost_func = np.conj(model_vis)*np.sum(
        cov_mat*(data_vis - gains1_expanded*np.conj(gains2_expanded)*model_vis)[:, :, np.newaxis],
        axis=2
    )
    cost_func = reformat_baselines_to_antenna_matrix(cost_func, gains_exp_mat_1, gains_exp_mat_2)
    cost_func_transpose = np.transpose(np.conj(cost_func), (1, 0, 2))
    freq_diagonals = -2*(cost_func + cost_func_transpose)
    for freq in range(Nfreqs):
        hess[:, :, freq, freq, 0] += np.real(freq_diagonals[:, :, freq])
        hess[:, :, freq, freq, 1] += np.imag(freq_diagonals[:, :, freq])
        hess[:, :, freq, freq, 2] += np.real(freq_diagonals[:, :, freq])
        
    if False: # Test reformatting
        hess_reformatted = np.zeros(
            (2, 2, Nants*Nfreqs, Nants*Nfreqs), dtype=float
        )
        hess_reformatted[0, 0, :, :] = hess[:, :, :, :, 0].reshape(
            Nants*Nfreqs, Nants*Nfreqs
        )
        hess_reformatted[0, 1, :, :] = hess[:, :, :, :, 1].reshape(
            Nants*Nfreqs, Nants*Nfreqs
        )
        hess_reformatted[1, 0, :, :] = (hess[:, :, :, :, 1].reshape(
            Nants*Nfreqs, Nants*Nfreqs)
        ).T
        hess_reformatted[1, 1, :, :] = hess[:, :, :, :, 2].reshape(
            Nants*Nfreqs, Nants*Nfreqs
        )
        del hess
        hess_reformatted = hess_reformatted.reshape(2*Nants*Nfreqs, 2*Nants*Nfreqs)

        hess_reformatted = hess_reformatted.reshape(2, 2, Nants, Nants, Nfreqs, Nfreqs)
        hess = np.zeros((Nants, Nants, Nfreqs, Nfreqs, 3), dtype=float)
        hess[:, :, :, :, 0] = hess_reformatted[0, 0, :, :, :, :]
        hess[:, :, :, :, 1] = hess_reformatted[0, 1, :, :, :, :]
        hess[:, :, :, :, 2] = hess_reformatted[1, 1, :, :, :, :]

    return hess

In [13]:
# Test real-real Hessian calculation
test_ant = 4
test_freq = 2
readout_ant = 5
readout_freq = 2
delta_gains = 0.0001
gains0 = np.copy(gains)
gains0[test_ant, test_freq] -= delta_gains/2.
gains1 = np.copy(gains)
gains1[test_ant, test_freq] += delta_gains/2.
grad0 = calculate_grad_real(gains_exp_mat_1, gains_exp_mat_2, gains0, data_vis, model_vis, cov_mat)
grad1 = calculate_grad_real(gains_exp_mat_1, gains_exp_mat_2, gains1, data_vis, model_vis, cov_mat)
hess = calculate_hess(gains_exp_mat_1, gains_exp_mat_2, gains, data_vis, model_vis, cov_mat, cal.Nants_data, cal.Nfreqs, data.Nblts)
print((grad1[readout_ant, readout_freq]-grad0[readout_ant, readout_freq])/delta_gains)
print(hess[test_ant, readout_ant, test_freq, readout_freq, 0])

786.2404964544112
786.2404964529857


In [14]:
# Test real-imag Hessian calculation
test_ant = 4
test_freq = 2
readout_ant = 5
readout_freq = 2
delta_gains = 0.0001
gains0 = np.copy(gains)
gains0[test_ant, test_freq] -= 1j*delta_gains/2.
gains1 = np.copy(gains)
gains1[test_ant, test_freq] += 1j*delta_gains/2.
grad0 = calculate_grad_real(gains_exp_mat_1, gains_exp_mat_2, gains0, data_vis, model_vis, cov_mat)
grad1 = calculate_grad_real(gains_exp_mat_1, gains_exp_mat_2, gains1, data_vis, model_vis, cov_mat)
hess = calculate_hess(gains_exp_mat_1, gains_exp_mat_2, gains, data_vis, model_vis, cov_mat, cal.Nants_data, cal.Nfreqs, data.Nblts)
print((grad1[readout_ant, readout_freq]-grad0[readout_ant, readout_freq])/delta_gains)
print(hess[test_ant, readout_ant, test_freq, readout_freq, 1])

70.42852439553826
70.42852439752825


In [15]:
# Test imag-imag Hessian calculation
test_ant = 4
test_freq = 2
readout_ant = 5
readout_freq = 2
delta_gains = 0.0001
gains0 = np.copy(gains)
gains0[test_ant, test_freq] -= 1j*delta_gains/2.
gains1 = np.copy(gains)
gains1[test_ant, test_freq] += 1j*delta_gains/2.
grad0 = calculate_grad_imag(gains_exp_mat_1, gains_exp_mat_2, gains0, data_vis, model_vis, cov_mat)
grad1 = calculate_grad_imag(gains_exp_mat_1, gains_exp_mat_2, gains1, data_vis, model_vis, cov_mat)
hess = calculate_hess(gains_exp_mat_1, gains_exp_mat_2, gains, data_vis, model_vis, cov_mat, cal.Nants_data, cal.Nfreqs, data.Nblts)
print((grad1[readout_ant, readout_freq]-grad0[readout_ant, readout_freq])/delta_gains)
print(hess[test_ant, readout_ant, test_freq, readout_freq, 2])

-612.2603700168838
-612.2603700065032
